In [1]:
import iris
import sys
import importlib
import numpy as np
from random import random, uniform
from copy import deepcopy

In [80]:
importlib.reload(iris)

<module 'iris' from 'E:\\REPOS\\int_sys\\swarm\\iris.py'>

In [2]:
class Particle:
    def __init__(self, X_train, y_train, min_w=-5.0, max_w=5.0, c1 = 1.49445, c2 = 1.49445, w = 0.729):
        self.min_w = min_w
        self.max_w = max_w
        self.c1 = c1
        self.c2 = c2
        self.w = w
        
        nn = iris.IrisNN(gen_randomly=True) # 4-6-3 neural net with randomly generated parameters
        self.position = nn.merge_params()     
        self.init_velocity()
        #self.velocity = np.random.uniform(self.min_w / 2.0, self.max_w / 2.0, size=len(self.position))
        self.fitness = nn.compute_mse(X_train, y_train)
        self.pbest = deepcopy([self.position, self.fitness])
    
    # t.random Microsoft guy
    def init_velocity(self):
        lo = 0.1 * self.min_w;
        hi = 0.1 * self.max_w;
        self.velocity = np.zeros(len(self.position))
        for i in range(0, len(self.position)):
            self.velocity[i] = (hi - lo) * random() + lo;
     
    def compute_fitness(self, X_train, y_train):
        nn = iris.IrisNN()
        nn.unpack_params(self.position)
        self.fitness = nn.compute_mse(X_train, y_train)
        if self.fitness < self.pbest[1]:
            self.pbest = deepcopy([self.position, self.fitness])
            
    def update(self, gbest):
        self.velocity = (self.w * self.velocity) \
                                      + (random() * self.c1 * (self.pbest[0] - self.position))\
                                      + (random() * self.c2 * (gbest[0] - self.position))
            
        self.position += self.velocity
        self.position[self.position < self.min_w] = self.min_w
        self.position[self.position > self.max_w] = self.max_w
        

In [3]:
class PSO:
    def __init__(self, swarm_size=12, epochs=700):
        if swarm_size < 1 or epochs < 0:
            print('Invalid input.Exiting..')
            sys.exit(-1)
        self.swarm_size = swarm_size
        self.epochs = epochs
        self.df = iris.IrisDF()
        self.gen_random_swarm()
    
    def gen_random_swarm(self):
        self.swarm = []
        self.swarm.append(Particle(self.df.X_train, self.df.y_train))
        self.gbest = self.swarm[0].pbest
        for i in range(1, self.swarm_size):
            p = Particle(self.df.X_train, self.df.y_train)
            if p.pbest[1] < self.gbest[1]:
                self.gbest = p.pbest
            self.swarm.append(p)
        
    def optimize(self):
        for i in range(0, self.epochs):
            for p in self.swarm:
                # update velocity and position
                p.update(self.gbest)
                # update fitness and eventualy pbest
                p.compute_fitness(self.df.X_train, self.df.y_train)
                # update gbest
                if p.pbest[1] < self.gbest[1]:
                    self.gbest = p.pbest
            if i % 100 == 0:
                print('Epoch {} with gbest mse of {}.'.format(i, self.gbest[1]))
                    
    def show_accuracy(self):
        nn = iris.IrisNN()
        nn.unpack_params(self.gbest[0])
        acc_train = nn.compute_accuracy(self.df.X_train, self.df.y_train)
        acc_test = nn.compute_accuracy(self.df.X_test, self.df.y_test)
        print("Accuracy on training set: {}".format(acc_train))
        print("Accuracy on test set: {}".format(acc_test))
    

In [24]:
pso = PSO()
pso.optimize()

Epoch 0 with gbest mse of 0.6708587315718885.
Epoch 100 with gbest mse of 0.2005500613720833.
Epoch 200 with gbest mse of 0.20025701335833246.
Epoch 300 with gbest mse of 0.20025666515741075.
Epoch 400 with gbest mse of 0.2002565402650984.
Epoch 500 with gbest mse of 0.20025653883955272.
Epoch 600 with gbest mse of 0.20025652889646983.


In [25]:
pso.show_accuracy()

Accuracy on training set: 0.9714285714285714
Accuracy on test set: 0.9555555555555556


In [26]:
for p in pso.swarm: print(p.pbest[1])

0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
0.200256528896
